In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from matplotlib import pyplot as plt

In [3]:
df = pd.read_csv("insurance_data.csv")
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['age','affordibility']],df.bought_insurance,test_size=0.2, random_state=25)

In [5]:
#scaling
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age'] / 100

In [7]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(2,), activation='sigmoid', kernel_initializer='ones', bias_initializer='zeros')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train_scaled, y_train, epochs=3000)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 5ms/step - loss: 0.6261 - accuracy: 0.6364
Epoch 487/3000
1/1 [==============================] - 0s 11ms/step - loss: 0.6261 - accuracy: 0.6364
Epoch 488/3000
1/1 [==============================] - 0s 6ms/step - loss: 0.6260 - accuracy: 0.6364
Epoch 489/3000
1/1 [==============================] - 0s 6ms/step - loss: 0.6260 - accuracy: 0.6364
Epoch 490/3000
1/1 [==============================] - 0s 6ms/step - loss: 0.6259 - accuracy: 0.6364
Epoch 491/3000
1/1 [==============================] - 0s 6ms/step - loss: 0.6259 - accuracy: 0.6364
Epoch 492/3000
1/1 [==============================] - 0s 6ms/step - loss: 0.6258 - accuracy: 0.6364
Epoch 493/3000
1/1 [==============================] - 0s 7ms/step - loss: 0.6257 - accuracy: 0.6364
Epoch 494/3000
1/1 [==============================] - 0s 10ms/step - loss: 0.6257 - accuracy: 0.6364
Epoch 495/3000
1/1 [==============================] - 0s 7ms/s

In [14]:
model.evaluate(X_test_scaled,y_test)

1/1 [==============================] - 0s 226ms/step - loss: 0.4515 - accuracy: 0.8333


[0.4515162706375122, 0.8333333134651184]

In [15]:
model.predict(X_test_scaled)

array([[0.67709583],
       [0.45932925],
       [0.25552067],
       [0.53705907],
       [0.6905662 ],
       [0.76434124]], dtype=float32)

In [16]:
coef, intercept = model.get_weights()

In [17]:
coef, intercept

(array([[3.1155188],
        [1.15558  ]], dtype=float32), array([-1.8794167], dtype=float32))

GRADIENT DESCENT FROM SCRATCH WITHOUT TENSORFLOW

In [8]:
#SIGMOID

def sigmoid_numpy(X):
   return 1/(1+np.exp(-X))

sigmoid_numpy(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

In [9]:
def log_loss(y_true, y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i,epsilon) for i in y_predicted]
    y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [10]:
def gradient_descent(age, affordability, y_true, epochs, loss_thresold):
    w1 = w2 = 1
    bias = 0
    rate = 0.5
    n = len(age)
    for i in range(epochs):
        weighted_sum = w1 * age + w2 * affordability + bias
        y_predicted = sigmoid_numpy(weighted_sum)
        loss = log_loss(y_true, y_predicted)

        w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true)) 
        w2d = (1/n)*np.dot(np.transpose(affordability),(y_predicted-y_true)) 

        bias_d = np.mean(y_predicted-y_true)
        w1 = w1 - rate * w1d
        w2 = w2 - rate * w2d
        bias = bias - rate * bias_d

        print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')

        if loss<=loss_thresold:
            break

    return w1, w2, bias


In [11]:
gradient_descent(X_train_scaled['age'],X_train_scaled['affordibility'],y_train, 1000, 0.4631)

Epoch:0, w1:0.974907633470177, w2:0.948348125394529, bias:-0.11341867736368583, loss:0.7113403233723417
Epoch:1, w1:0.9556229728273669, w2:0.9058873696677865, bias:-0.2122349122718517, loss:0.681264778737757
Epoch:2, w1:0.9416488476693794, w2:0.8719790823960313, bias:-0.2977578997796538, loss:0.6591474252715025
Epoch:3, w1:0.9323916996249162, w2:0.8457541517722915, bias:-0.3715094724003511, loss:0.6431523291301917
Epoch:4, w1:0.9272267472726993, w2:0.8262362885332687, bias:-0.43506643026891584, loss:0.6316873063379158
Epoch:5, w1:0.9255469396815343, w2:0.8124402814952774, bias:-0.48994490058938817, loss:0.623471707997592
Epoch:6, w1:0.9267936114129968, w2:0.8034375029757677, bias:-0.5375299543522853, loss:0.6175321183044205
Epoch:7, w1:0.93047170420295, w2:0.7983920007454487, bias:-0.5790424270894963, loss:0.6131591858705934
Epoch:8, w1:0.9361540784567942, w2:0.7965748796787705, bias:-0.6155315088627655, loss:0.6098518179750948
Epoch:9, w1:0.9434791243557357, w2:0.7973647616854131, bia

(5.051047623653049, 1.4569794548473887, -2.9596534546250037)

In [18]:
coef, intercept

(array([[3.1155188],
        [1.15558  ]], dtype=float32), array([-1.8794167], dtype=float32))